In [55]:
import pandas as pd
import re
import json


In [62]:
class Tariff:
    def __init__(self):
        self.CN_tariff_list = [
            {
                "heading": "9903.88.01",
                "duty": 25,
            },
            {
                "heading": "9903.88.02",
                "duty": 25,
            },
            {
                "heading": "9903.88.03",
                "duty": 25,
            },
            {
                "heading": "9903.88.15",
                "duty": 7.5,
            },
        ]

    def get_CN_tariff(self, heading):
        for tariff in self.CN_tariff_list:
            if tariff.get("heading") == heading:
                return tariff.get("duty")
        return 0  # no China tariff found


class Product:
    def __init__(self, dict):
        self.htsno = dict.get("htsno")
        self.indent = dict.get("indent")
        self.description = dict.get("description")
        self.units = dict.get("units")
        self.general = dict.get("general")
        self.special = dict.get("special")
        self.other = dict.get("other")
        self.footnotes = dict.get("footnotes")
        self.quotaQuantity = dict.get("quotaQuantity")
        self.additionalDuties = dict.get("additionalDuties")

        self.CN_subheading, self.CN_tariff = self.extract_china_tariff()

    def extract_china_tariff(self):
        tariffs = Tariff()

        try:
            # try to extract footnote
            footnote_text = self.footnotes[0].get("value")
            heading = re.findall(r"9903\.88\.\d{2}", footnote_text)[0]
            return (heading, tariffs.get_CN_tariff(heading))
        except:
            # no footnote found
            return (None, 0)

In [64]:
with open("htsdata/hts_2023_revision_11_json.json", "r") as htsjson:
    htsdata = json.load(htsjson)
    product_list = [Product(prod) for prod in htsdata]

htsdf = pd.DataFrame([prod.__dict__ for prod in product_list])


In [144]:
columns = ["htsno", "description", "general", "special", "CN_subheading", "CN_tariff"]
hts8df = htsdf.loc[htsdf["htsno"].str.len() == 10][columns] # keep 8-digit entries

hts8df["hts8"] = hts8df["htsno"].str.extract(r"(\d{4}\.\d{2}\.\d{2})").replace("\.", "", regex=True)
hts8df["hts6"] = hts8df["htsno"].str.extract(r"(\d{4}\.\d{2})\.?\d{2}").replace("\.", "", regex=True)

hts8df.to_csv("outputs/hts8_2023.csv", index=False)
hts8df

,htsno,description,general,special,CN_subheading,CN_tariff,hts8,hts6
2,0101.21.00,Purebred breeding animals,Free,,9903.88.15,7.5,01012100,010121
5,0101.29.00,Other,Free,,9903.88.15,7.5,01012900,010129
14,0102.21.00,Purebred breeding animals,Free,,9903.88.15,7.5,01022100,010221
22,0102.29.20,Cows imported specially for dairy purposes,Free,,9903.88.15,7.5,01022920,010229
25,0102.29.40,Other,1¢/kg,"Free (A+,AU,BH,CL,CO,D,E,IL,JO,KR,MA,OM,P,PA,P...",9903.88.15,7.5,01022940,010229
...,...,...,...,...,...,...,...,...
35591,9922.52.08,Goods of Canada provided for in note 2(b) to t...,,10.5¢/kg (S+),None,0.0,99225208,992252
35593,9922.52.09,Goods of Mexico or goods of the United States ...,,Free (S+),None,0.0,99225209,992252
35594,9922.52.10,Goods of Canada provided for in note 2(b) to t...,,2.6¢/kg (S+),None,0.0,99225210,992252
35596,9922.52.11,Goods of Mexico or goods of the United States ...,,Free (S+),None,0.0,99225211,992252
